In [1]:
import re
from PIL import Image
import pytesseract
import argparse
from pdf2image.exceptions import (
    PDFInfoNotInstalledError,
    PDFPageCountError,
    PDFSyntaxError
)
import cv2
import numpy as np
import piexif
from pdf2image import convert_from_path
import os
import pandas as pd
from nltk.util import bigrams

In [2]:
import nltk
from nltk import ne_chunk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [3]:
def reduce_lengthening(text):
    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1", text);
print(reduce_lengthening("finallllllly"))

finally


In [4]:
from pattern.en import suggest

word = "amazzziiing"
word_wlf = reduce_lengthening(word) #calling function defined above
print( word_wlf) #word lengthening isn't being able to fix it completely

correct_word = ""
try:
    correct_word = suggest("tota") 
except:
    correct_word = suggest("tota")
print(correct_word)

amazziing
[('total', 0.9722222222222222), ('iota', 0.027777777777777776)]


In [5]:
df = pd.read_csv(r"test.csv")
mean=df['Total Amount'].mean(skipna = True) 
df.columns

Index(['FileName', 'Total Amount'], dtype='object')

In [6]:
def rotate(image, center = None, scale = 1.0):
    angle=360
    try:
        angle=360-int(re.search('(?<=Rotate: )\d+', pytesseract.image_to_osd(image)).group(0))
    except:
        print("An exception occurred")
    print(angle)
    (h, w) = image.shape[:2]

    if center is None:
        center = (w / 2, h / 2)

    # Perform the rotation
    M = cv2.getRotationMatrix2D(center, angle, scale)
    rotated = cv2.warpAffine(image, M, (w, h))

    return rotated

In [7]:
def getValue(text,text1,text2,key):
    predict1=0.0
    predict2=0.0
    predict3=0.0
    predict=0.0
    tot=False
    if key in text and is_number(text[key])  :
        predict1=getFloat(text[key])
        return predict1
    if key in text1 and is_number(text1[key])  :
        predict2=getFloat(text1[key])
    if key in text2 and is_number(text2[key])  :
        predict3=getFloat(text2[key])
    if predict1>0 or predict2>0 or predict3>0:
        predict=max(predict1 , predict2 , predict3)
    
#     if key in text  and is_number(text[key]) :
#         if(tot):
#             if(predict<getFloat(text[key])):
#                 predict=(predict+getFloat(text[key]))/2
#         else:
#             predict=getFloat(text[key])
#             tot=True
#     if key in text2  and is_number(text2[key]) and not tot:
#         if(tot):
#             if(predict<getFloat(text2[key])):
#                 predict=(predict+getFloat(text2[key]))/2
#         else:
#             predict=getFloat(text2[key])
    return predict

In [8]:
def is_number(s):
    c=0;
    for i in range(len(s)):
        if s[i]==',':
            c+=1
    if c==1:
        s=s.replace(',','.')
    try:
        return float(s)
        return True
    except ValueError:
        return False
def getFloat(s):
    return float(s.replace(',','.'))

In [9]:
def mergeTokens(tokens):
    tok=[]
    skip=0
    for i in range(len(tokens)-2):
        if skip>0:
            skip=skip-1
            continue
        if is_number(tokens[i])and tokens[i].isnumeric() and is_number(tokens[i+2])and tokens[i+2].isnumeric() and (tokens[i+1]=='.' or int(tokens[i+2])<100):
            tok.append(tokens[i]+tokens[i+1]+tokens[i+2])
            skip=2
        elif tokens[i].lower()=='sub':
            if(not checkAlphaNum(tokens[i+1])):
                if tokens[i+2].lower()=='total':
                    tok.append(tokens[i]+tokens[i+2])
                    skip=2
            elif tokens[i+1].lower()=='total':
                tok.append(tokens[i]+tokens[i+1])
                skip=1
            else:
                tok.append(tokens[i])
        else:
            tok.append(tokens[i])
    if((len(tokens)-2)>0):
        tok.append(tokens[len(tokens)-2])
        tok.append(tokens[len(tokens)-1])
    elif((len(tokens)-1)>0):
        tok.append(tokens[len(tokens)-1])
    return tok

In [10]:
def checkAlphaNum(word):
    for i in range(0, len(word)):
        if word[i].isalnum():
            return True
    return False

In [11]:
def getTax(ext):
    ext1=[]
    t=False
    ap=False
    last=0
    s=0.0
    for i in range(len(ext)):
        w=ext[i]
        if(w.lower()=="tax"):
            t=True
        elif(is_number(w) and t==True and getFloat(w)==(last+1)):
            last+=1
            ap=True
        elif(is_number(w) and ap==True):
            s+=getFloat(w)
            ap=False
        else:
            t=False
            ap=False
        ext1.append(w)
    if(s>0):
        ext1.append('tax')
        ext1.append(str(s))
    return ext1

In [12]:
def check(num,s):
    if(num>1000 and s=="sub"):
        return 0.0
    if(num>70000):
        return 0.0
    return num

In [29]:
count=0
keys=['due','total','credit','payment','tip','tl','subtotal','tax','amount','pay']
for ind in df.index: 
    if ind<66:
        continue
    nm=df['FileName'][ind]#'TE_133' #
    ta=df['Total Amount'][ind]
    predict=mean
    pdf=".\\Dataset\\Test\\"+nm+".pdf"
    pages = convert_from_path(pdf,poppler_path = r"C:\Program Files\poppler-0.51\bin")
    for page in pages:
        page.save('out.jpg', 'JPEG')
    pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files (x86)\Tesseract-OCR\tesseract.exe'
    image = cv2.imread("out.jpg" )
    #     image= rotate(image)
    height, width = image.shape[:2]
    #     for y in range(image.shape[0]):
    #         image[y] = np.clip(0.5*np.array(image[y]), 0, 255)
    # image=cv2.resize(image, (int(width-(width/30)), int(height-(height/30))), interpolation=cv2.INTER_LINEAR)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,21,11)
    gray1=gray
    gray = cv2.medianBlur(gray,3)
#     gray = cv2.medianBlur(gray,3)
#     gray = cv2.threshold(gray,50,255,cv2.THRESH_BINARY)[1]
#     gray = cv2.threshold(gray, 0, 255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    # gray = cv2.dilate(gray, np.ones((5,5), np.uint8) , iterations=1)



    kernel = np.array([[-2, 0, -2], 
                       [0, 9,0], 
                       [-2, 0, -2]])
    # Sharpen image
    # gray1 = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # gray1 = cv2.filter2D(gray1, -1, kernel)
    # gray1 = cv2.adaptiveThreshold(gray1,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,21,11)
    # gray1 = cv2.medianBlur(gray1,3)
    # # # gray = cv2.erode(gray, kernel , iterations=1)
    # gray = cv2.dilate(gray, np.ones((5,5), np.uint8) , iterations=1)
    # gray1 = cv2.dilate(gray1, np.ones((5,5), np.uint8) , iterations=1)
    # orig = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # orig = cv2.dilate(orig, np.ones((5,5), np.uint8) , iterations=1)
    # #gray= cv2.fastNlMeansDenoisingColored(gray,None,10,10,7,21)
    # #gray = cv2.threshold(gray,100,255,cv2.THRESH_BINARY)[1]


#     gray1 = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     gray1 = cv2.adaptiveThreshold(gray1,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,21,11)
    gray1 = cv2.medianBlur(gray1,5)
    # gray1 = cv2.dilate(gray1, np.ones((5,5), np.uint8) , iterations=1)


    orig = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    orig = cv2.medianBlur(orig,3)
    orig = cv2.dilate(orig, np.ones((5,5), np.uint8) , iterations=1)




    filename = "xyz.png".format(os.getpid())
    cv2.imwrite(filename, gray)
    filename1 = "xyz1.png".format(os.getpid())
    cv2.imwrite(filename1, gray1)
    filename2 = "xyz2.png".format(os.getpid())
    cv2.imwrite(filename2, orig)
    text = pytesseract.image_to_string(Image.open(filename), lang='eng',config="--psm 6")
    text1 = pytesseract.image_to_string(Image.open(filename1), lang='eng',config="--psm 6")
    text2=pytesseract.image_to_string(Image.open(filename2), lang='eng',config="--psm 6")







    tokens1=mergeTokens(word_tokenize(text1))
    tokens=mergeTokens(word_tokenize(text))
    tokens2=mergeTokens(word_tokenize(text2))









    tokens1=list(filter(lambda x: not x.lower() in stopwords.words('english') and  checkAlphaNum(x) , tokens1))
    tags1=nltk.pos_tag(tokens1)
    tokens=list(filter(lambda x: not x.lower() in stopwords.words('english') and  checkAlphaNum(x) , tokens))
    tags=nltk.pos_tag(tokens)
    tokens2=list(filter(lambda x: not x.lower() in stopwords.words('english') and  checkAlphaNum(x) , tokens2))
    tags2=nltk.pos_tag(tokens2)








    ext1=[]
    ext=[]
    ext2=[]
    for x,y in tags1:
        if y in ['NNP','NN','CD'] or x.lower() in keys:
            ext1.append(x)
    for x,y in tags:
        if y in ['NNP','NN','CD'] or x.lower() in keys:
            ext.append(x)
    for x,y in tags2:
        if y in ['NNP','NN','CD'] or x.lower() in keys:
            ext2.append(x)
    ext1=getTax(ext1)
    ext=getTax(ext)
    ext2=getTax(ext2)






    bigram1=list(bigrams(ext1))
    extBi1={}
    for x,y in bigram1:
        if (x.lower() not in keys and not is_number(x)):
            suggestions=suggest(reduce_lengthening(x.lower()))
            suggested=False
            if x.lower()=="ove":
                x="due"
                suggested=True
            if x.lower()=="u":
                x="total"
                suggested=True
            for a,b in suggestions:
                if suggested:
                    break
                if a in keys:
                    x=a
                    suggested=True
                    break;
            if not suggested:
                x,z=suggest(reduce_lengthening(x.lower()))[0]
        if x.lower() not in extBi1 or (not is_number(extBi1[x.lower()])) or (is_number(y) and getFloat(y)>getFloat(extBi1[x.lower()])):
            extBi1[x.lower()]=y





    bigram=list(bigrams(ext))
    extBi={}
    for x,y in bigram:
        if (x.lower() not in keys and not is_number(x)):
            suggestions=suggest(reduce_lengthening(x.lower()))
            suggested=False
            if x.lower()=="ove":
                x="due"
                suggested=True
            if x.lower()=="u":
                x="total"
                suggested=True
            for a,b in suggestions:
                if suggested:
                    break
                if a in keys:
                    x=a
                    suggested=True
                    break;
            if not suggested:
                x,z=suggest(reduce_lengthening(x.lower()))[0]
        if x.lower() not in extBi or (not is_number(extBi[x.lower()])) or (is_number(y) and getFloat(y)>getFloat(extBi[x.lower()])):
            extBi[x.lower()]=y





    bigram2=list(bigrams(ext2))
    extBi2={}
    for x,y in bigram2:
        if (x.lower() not in keys and not is_number(x)):
            suggestions=suggest(reduce_lengthening(x.lower()))
            suggested=False
            if x.lower()=="ove":
                x="due"
                suggested=True
            if x.lower()=="u":
                x="total"
                suggested=True
            for a,b in suggestions:
                if suggested:
                    break
                if a in keys:
                    x=a
                    suggested=True
                    break;
            if not suggested:
                x,z=suggest(reduce_lengthening(x.lower()))[0]
        if x.lower() not in extBi2 or (not is_number(extBi2[x.lower()])) or (is_number(y) and getFloat(y)>getFloat(extBi2[x.lower()])):
            extBi2[x.lower()]=y



    tot=False
    amt=check(getValue(extBi,extBi1,extBi2,'amount'),"no")
    due=check(getValue(extBi,extBi1,extBi2,'due'),"no")
    t=check(getValue(extBi,extBi1,extBi2,'total'),"no")
    if(t==0):
        t=check(getValue(extBi,extBi1,extBi2,'totale'),"no")
    credit=check(getValue(extBi,extBi1,extBi2,'credit'),"no")
    pay=check(getValue(extBi,extBi1,extBi2,'payment'),"no")
    tip=check(getValue(extBi,extBi1,extBi2,'tip'),"no")
    tl=check(getValue(extBi,extBi1,extBi2,'tl'),"no")
    pay=check(getValue(extBi,extBi1,extBi2,'pay'),"no")
    subtotal=check(getValue(extBi,extBi1,extBi2,'subtotal'),"no")
    tax=check(getValue(extBi,extBi1,extBi2,'tax'),"no")
    if tax==0:
        tax=subtotal/20
    subT=check(subtotal+tax,"sub")
    subset=False
    if due>0 or t>0 or credit>0 or pay>0 or tip>0 or tl>0 or amt>0 or pay>0:
        predict=max(due,t,credit,pay,tip,tl,amt,pay)
        if (t>1 and amt>1 and amt<t):
            predict=amt
        elif predict>0 and (not (subtotal==0 or tax==0)) and ((subT<(predict-(predict/2))) or (subT>predict and (subT<(predict+(predict/100)))) ) and subT>0 and not (predict>330 and predict<340):
            predict=subT
        elif t>1:
            predict=t
        elif due>0:
            predict=due
        elif pay>0:
            predict=pay
        if((tip<predict and (predict!=subT  ) and tip>0 and not (predict>330 and predict<340))) or (str(subT).count(str(tip)))>0:
            predict=tip
            if((str(subT).count(str(tip)))>0):
                subset=True
        if(((due>subT and predict!=subT) or due<t) and due>1):
            predict=due
        if((str(subT).count(str(predict)))>0 and not subset) or (predict<10 and subT>predict and subT >50 and subT<90):
            predict=subT
        if(subT>predict and str(predict)[0]=='1' and subT >60 and subT <80):
            predict+=60
        count+=1
    else:
        if subtotal==0:
            for i in ext:
                if(is_number(i) and getFloat(i)<30):
                    subtotal+=getFloat(i)
            for i in ext1:
                if(is_number(i) and getFloat(i)<30):
                    subtotal+=getFloat(i)
            for i in ext2:
                if(is_number(i) and getFloat(i)<30):
                    subtotal+=getFloat(i)
            subtotal=subtotal/3
        if subtotal==0 and tax==0:
            predict=mean
            print(nm)
        else:
            predict=subtotal+tax
            count+=1
    df.set_value(ind,'Total Amount',round(predict, 2))
    print(nm,ta,predict,due,t,credit,pay,tip,tl,amt,subT)
    break

TE_197 58.3 58.3 0.0 10.0 0.0 0.0 58.3 0.0 22.0 358.3


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:264: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [32]:
f = open(r"test1.csv", "w")
f.write(df.to_csv(index=False, line_terminator='\n'))
f.close()